In [1]:
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn.objects as so
from seaborn import axes_style

from IPython.display import display

In [2]:
gff3_col = [
    "seqid",
    "source",
    "type",
    "start",
    "end",
    "score",
    "strand",
    "phase",
    "attributes",
    "irl read number",
    "irr read number",
    "norm irl",
    "norm irr",
    "SampleID",
    "treatment",
    "mouseID",
]

insertion_dir = Path("/home/fisch872/mat/projects/Laura-SB-Analysis/2020_SB-output/GRCm39/results-insertions")


In [43]:
insert_list = []
for file in insertion_dir.iterdir():
    tmp_df = pd.read_csv(file, sep="\t")
    tmp_meta = file.name.split(".")[0].split("-")
    if len(tmp_meta) == 3:  # 2020 SB
        tmp_df["treatment"] = tmp_meta[2]
        tmp_df["sampleID"] = tmp_meta[1]
        tmp_df["cell_type"] = tmp_meta[0]
    elif len(tmp_meta) == 2:  # 2023 SB
        tmp_df["treatment"] = tmp_meta[0]
        tmp_df["sampleID"] = tmp_meta[1]
    tmp_df["sample_sub_id"] = range(len(tmp_df))
    insert_list.append(tmp_df)
inserts_df = pd.concat(insert_list, ignore_index=True)
display(inserts_df)

,chr,pos,strand,ref_length,query_length,read_length,mapping_quality,read_name,TA_location,read_first_last,ref_first_last,tpn_promoter_orient,library,treatment,sampleID,cell_type,sample_sub_id
0,NC_000067.7,53601099,+,135,147,147,14,K00274:76:HHHM2BBXX:8:2128:16224:28446,none,GATAAATTTG-AGGTCCTAGA,GGAGcAGTTG-AGGTCCTAGA,-,IRL,LT,32_2,EL4,0
1,NC_000067.7,68392290,+,70,71,71,42,K00274:76:HHHM2BBXX:8:1108:29193:25984,first,TATATATAGA-GCCTCATTTT,TATATATAGA-TGCCTCATTT,+,IRR,LT,32_2,EL4,1
2,NC_000067.7,68392290,+,71,71,71,42,K00274:76:HHHM2BBXX:8:1227:5974:17685,first,TATATATAGA-GCCTCATTTC,TATATATAGA-GCCTCATTTC,+,IRR,LT,32_2,EL4,2
3,NC_000067.7,68392290,+,68,68,68,28,K00274:76:HHHM2BBXX:8:2108:24870:26265,first,TATATATAGA-TCTGCCTCAT,TATATATAGA-TCTGCCTCAT,+,IRR,LT,32_2,EL4,3
4,NC_000067.7,68392290,+,67,67,67,28,K00274:76:HHHM2BBXX:8:2202:3133:47084,first,TATATATAGA-TTCTGCCTCA,TATATATAGA-TTCTGCCTCA,+,IRR,LT,32_2,EL4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242482,NC_000083.7,74940171,-,51,67,67,22,K00275:54:HHHJTBBXX:8:2118:20811:11530,first,TATAGGGATC-TCCCTTAAGC,CCTTCATGGT-TAGGGGATCC,-,IRR,LT,456_4,B16,6
242483,NC_000085.7,55151157,-,134,148,148,18,K00275:54:HHHJTBBXX:8:1109:25986:35936,first,TAATACGACT-TGTGTGTTTG,CAAACACACA-GGAGCCTATA,-,IRR,LT,456_4,B16,7
242484,NC_000072.7,86910728,+,102,146,146,41,K00274:188:HW2KTBBXX:5:2125:20557:21852,none,TTTTGTAAAC-CACCAGTGCT,TAACTAACAA-CACCAGTGCT,+,IRR,RT,46_2,EL4,0
242485,NC_000072.7,86910821,-,108,148,148,41,K00274:188:HW2KTBBXX:5:1101:6948:19619,none,GTGGATTAAA-ATAGTGAGTC,ACCAGTGCTG-ACAATTCCTG,+,IRL,RT,46_2,EL4,1


In [54]:
out_df = pd.DataFrame(inserts_df["chr"])
out_df.columns = ["seqid"]
out_df["source"] = "T2/Onc3"
out_df["type"] = "insertion site"
out_df["start"] = inserts_df["pos"]
out_df["end"] = inserts_df["pos"]
out_df["score"] = inserts_df["mapping_quality"]
out_df["strand"] = inserts_df["strand"]
out_df["phase"] = "."
out_df["attributes"] = inserts_df.apply(lambda x: f"ID={x['treatment']}:{x['sampleID']}*{x['sample_sub_id']};color=#{'154360' if x['tpn_promoter_orient'] == '+' else 'D35400'}", axis=1)
# strand is still the strand that the insertion was on
# tpn_promoter_orient is the orientation w.r.t. the IRL or IRR library
# the color in attributes is based on tpn_promoter_orient
out_df = pd.concat((out_df, inserts_df.iloc[:, 11:-1]), axis=1)
out_df.to_csv(insertion_dir.parent / "all_insertions.gff3", sep="\t", header=True, index=False)
display(out_df)

,seqid,source,type,start,end,score,strand,phase,attributes,tpn_promoter_orient,library,treatment,sampleID,cell_type
0,NC_000067.7,T2/Onc3,insertion site,53601099,53601099,14,+,.,ID=LT:32_2*0;color=#D35400,-,IRL,LT,32_2,EL4
1,NC_000067.7,T2/Onc3,insertion site,68392290,68392290,42,+,.,ID=LT:32_2*1;color=#154360,+,IRR,LT,32_2,EL4
2,NC_000067.7,T2/Onc3,insertion site,68392290,68392290,42,+,.,ID=LT:32_2*2;color=#154360,+,IRR,LT,32_2,EL4
3,NC_000067.7,T2/Onc3,insertion site,68392290,68392290,28,+,.,ID=LT:32_2*3;color=#154360,+,IRR,LT,32_2,EL4
4,NC_000067.7,T2/Onc3,insertion site,68392290,68392290,28,+,.,ID=LT:32_2*4;color=#154360,+,IRR,LT,32_2,EL4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242482,NC_000083.7,T2/Onc3,insertion site,74940171,74940171,22,-,.,ID=LT:456_4*6;color=#D35400,-,IRR,LT,456_4,B16
242483,NC_000085.7,T2/Onc3,insertion site,55151157,55151157,18,-,.,ID=LT:456_4*7;color=#D35400,-,IRR,LT,456_4,B16
242484,NC_000072.7,T2/Onc3,insertion site,86910728,86910728,41,+,.,ID=RT:46_2*0;color=#154360,+,IRR,RT,46_2,EL4
242485,NC_000072.7,T2/Onc3,insertion site,86910821,86910821,41,-,.,ID=RT:46_2*1;color=#154360,+,IRL,RT,46_2,EL4
